In [1]:
use mavenfuzzyfactory

Commands completed successfully

Total execution time: 00:00:00

In [3]:
select * from website_pageviews limit 10

(10 row(s) affected)

Total execution time: 00:00:01.005

website_pageview_id,created_at,website_session_id,pageview_url
1,2012-03-19 08:04:16,1,/home
2,2012-03-19 08:16:49,2,/home
3,2012-03-19 08:26:55,3,/home
4,2012-03-19 08:37:33,4,/home
5,2012-03-19 09:00:55,5,/home
6,2012-03-19 09:05:46,6,/home
7,2012-03-19 09:06:27,7,/home
8,2012-03-19 09:10:08,6,/products
9,2012-03-19 09:10:52,6,/the-original-mr-fuzzy
10,2012-03-19 09:14:02,6,/cart


**Subject: Top Website Pages:** Could you help me get my head around the site by pulling <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">the most-viewed website pages, ranked by session volume?</span>

In [10]:
select pageview_url, count(DISTINCT website_session_id) as session
from website_pageviews
where created_at < '2012-06-09'
GROUP BY pageview_url
ORDER BY session desc
limit 10

(7 row(s) affected)

Total execution time: 00:00:01.417

pageview_url,session
/home,10403
/products,4239
/the-original-mr-fuzzy,3037
/cart,1306
/shipping,869
/billing,716
/thank-you-for-your-order,306


**Subject: Top Entry Pages -** Would you be able to pull a list of the top entry pages? I <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">want to confirm where our users are hitting the site.&nbsp;</span>  <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">If you could pull all entry pages and rank them on entry&nbsp;</span>  <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">volume, that would be great.</span>

In [41]:
-- creating a temporary table for first_entry_page later to be used.
CREATE TEMPORARY TABLE first_pageview
SELECT website_session_id, min(website_pageview_id) as first_entry_page
FROM website_pageviews
where created_at < '2012-06-12'
GROUP by website_session_id

Commands completed successfully

Total execution time: 00:00:04.031

In [40]:
drop TEMPORARY table first_pageview

Commands completed successfully

Total execution time: 00:00:00.050

In [45]:
select * from first_pageview limit 10

(10 row(s) affected)

Total execution time: 00:00:01.003

website_session_id,first_entry_page
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,12
9,14
10,15


In [50]:
-- IMPORTANT: 
-- LEFT TABLE will be the View we created it's logical to have info for these only
-- We have to join on pageviews it will contain info about more pageview to count session
select website_pageviews.pageview_url, count(DISTINCT first_pageview.website_session_id) as sessions_hitting_this_landing_page
from first_pageview LEFT JOIN website_pageviews
on website_pageviews.website_pageview_id = first_pageview.first_entry_page
where created_at < '2012-06-12'
GROUP BY pageview_url
order by 2 desc

(1 row(s) affected)

Total execution time: 00:00:01.042

pageview_url,sessions_hitting_this_landing_page
/home,10714


In [ ]:
-- BUSINESS CONTEXT: we want to see landing page performance for a certain time period
-- STEP 1: find the first website_pageview_id for relevant sessions
-- STEP 2: identify the landing page of each session
-- STEP 3: counting pageviews for each session, to identify "bounces"
-- STEP 4: summarizing total sessions and bounced sessions, by LP